In [1]:
import pandas as pd
import rpy2
import readline
import numpy as np
from scipy import stats as scistats
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import re

%matplotlib inline

%load_ext rpy2.ipython

In [2]:
def prettify_ax(ax):
    """
    Nifty function we can use to make our axes more pleasant to look at
    """
    for spine in ax.spines.itervalues():
        spine.set_visible(False)
    ax.set_frameon=True
    ax.patch.set_facecolor('#eeeeef')
    ax.grid('on', color='w', linestyle='-', linewidth=1)
    ax.tick_params(direction='out')
    ax.set_axisbelow(True)
    
def simple_ax(figsize=(6,4), **kwargs):
    """
    Shortcut to make and 'prettify' a simple figure with 1 axis
    """
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111, **kwargs)
    prettify_ax(ax)
    return fig, ax

# Counts -> TMM Normalized cpm

In [7]:
def log_cpm(counts, libraries):
    log_counts = pd.DataFrame()
    for col in counts:
        #R_col = sum(counts[col])
        #R_eff_coll = R_col * libraries.ix[col, 'norm.factors']
        #counts[col] = counts[col].apply(lambda x: (x + 0.5) / ((R_col + 1.0) / (10 ** 6)))
        #counts[col] = counts[col].apply(lambda x: np.log2(x))
        R_eff = libs.ix[col, 'effective.lib.size']
        log_counts[col] = np.log2(1e6*(counts[col]+0.5)/(R_eff + 1))
    return log_counts

def tmm_norm(mat):
    %Rpush mat
    %R library(edgeR)
    %R library(limma)
    %R myDGEList <- DGEList(counts=mat)
    %R x <- calcNormFactors(myDGEList)
    %R calcNormFactorsResults <- x[['samples']]
    %R rowNames <- rownames(calcNormFactorsResults)
    %Rpull calcNormFactorsResults
    %Rpull rowNames
    libraries = calcNormFactorsResults.set_index(rowNames)
    libraries['effective.lib.size'] = libraries['lib.size'] * libraries['norm.factors']
    return libraries

# PCA

In [13]:
def project(mean_centered, num_pcs):
    XTX = (1.0 / len(mean_centered.T)) * mean_centered.T.dot(mean_centered)
    evals, evecs = np.linalg.eigh(XTX)
    indices = (-evals).argsort()
    D = np.diag(evals[indices])
    V = []
    for ev in evecs:
        V.append(ev[indices])
    V = np.array(V)
    D_R = D[:num_pcs, :num_pcs] ** (-0.5)
    D_R[D_R == np.inf] = 0
    SVD = np.sqrt(1.0 / len(mean_centered.T)) * mean_centered.dot(V[:, :num_pcs].dot(D_R))
    P = np.array(SVD.T.dot(mean_centered))
    return evals, P, indices

def plot_pcs(P, idx1, idx2, c=None):
    fig, ax = simple_ax(figsize=(6,6))
    ax.scatter(P[idx1], P[idx2], c=c, s=100, edgecolor='w')

    ax.set_title('PC1 vs PC2')
    ax.set_xlabel('Principal Component ' + str(idx1 + 1))
    ax.set_ylabel('Principal Component ' + str(idx2 + 1))
    
def plot_var(eigenvalues, indices, xlim):
    tot_var = eigenvalues[indices].sum()
    pc_vars = eigenvalues[indices] / tot_var

    fig, ax = simple_ax(figsize=(6,6))
    ax.plot(pc_vars, 'o-')
    ax.set_xlim([1, xlim])
    ax.set_xticks(range(xlim))
    ax.set_xticklabels(range(1, xlim))

    ax.set_title('Percentage of variance captured by Principal Components')
    ax.set_xlabel('Principal Component')
    ax.set_ylabel('Percentage of variance captured')

# Clustering

In [3]:
import fastcluster
from sklearn import cluster, datasets
import scipy.cluster.hierarchy as hier
import matplotlib.colors as mcolors

def linkage(mat):
    gene_link = fastcluster.linkage(mat, method='ward', metric='correlation')
    cell_link = fastcluster.linkage(mat.T, method='ward', metric='correlation')

    gene_order = hier.leaves_list(gene_link)
    cell_order = hier.leaves_list(cell_link)

    reordered_data_subset = mat.values[gene_order, :]
    reordered_data_subset = reordered_data_subset[:, cell_order]
    #reordered_data_subset = log2_cpm.ix[gene_order, cell_order]
    return gene_link, cell_link, reordered_data_subset

# From: http://stackoverflow.com/questions/16834861/create-own-colormap-using-matplotlib-and-plot-color-scale
def make_colormap(seq):
    """Return a LinearSegmentedColormap
    seq: a sequence of floats and RGB-tuples. The floats should be increasing
    and in the interval (0,1).
    """
    seq = [(None,) * 3, 0.0] + list(seq) + [1.0, (None,) * 3]
    cdict = {'red': [], 'green': [], 'blue': []}
    for i, item in enumerate(seq):
        if isinstance(item, float):
            r1, g1, b1 = seq[i - 1]
            r2, g2, b2 = seq[i + 1]
            cdict['red'].append([item, r1, r2])
            cdict['green'].append([item, g1, g2])
            cdict['blue'].append([item, b1, b2])
    return mcolors.LinearSegmentedColormap('CustomMap', cdict)

def heatmap(gene_link, cell_link, reordered_data_subset):
    fig = plt.figure(figsize=(16, 8), dpi=96)
    from matplotlib import gridspec
    gs = gridspec.GridSpec(2, 2,
                           width_ratios=[1,10],
                           height_ratios=[1,10],
                           wspace=0.05,
                           hspace=0.05)


    # # x ywidth height
    ax1 = fig.add_subplot(gs[0,1], frameon=False)
    Z1 = hier.dendrogram(cell_link, orientation='top', ax=ax1) # adding/removing the axes
    ax1.set_xticks([])
    ax1.set_yticks([])

    ax2 = fig.add_subplot(gs[1,0], frameon=False)
    Z2 = hier.dendrogram(gene_link, orientation='left', ax=ax2) # adding/removing the axes
    ax2.set_xticks([])
    ax2.set_yticks([])

    axmatrix = fig.add_subplot(gs[1,1])
    # norm=mpl.colors.Normalize(vmin=0, vmax=40)
    c = mcolors.ColorConverter().to_rgb
    gbr = make_colormap(
            [c('black'), c('green'), 0.33, c('green'), c('red'), 0.66, c('red')])
    im = axmatrix.matshow(reordered_data_subset.T, aspect='auto', origin='lower', cmap=gbr, interpolation='none')
    axmatrix.set_xticks([])
    axmatrix.set_yticks([])

In [3]:
full_count = pd.read_csv('/n/regal/scrb152/Data/Yu_et_al/full_counts_matrix.csv')
fullCounts = full_count.iloc[:, 1:]
# Remove rows with all 0 counts
filtered_fullCounts = fullCounts.loc[(fullCounts!=0).any(axis=1)]
full_count.rename(columns={"Unnamed: 0": "Gene_id"}, inplace=True)
full_count.set_index('Gene_id', inplace=True)
genes = full_count.index
full_count

,Adr_F_002_1,Adr_F_002_2,Adr_F_002_3,Adr_F_002_4,Adr_F_006_1,Adr_F_006_2,Adr_F_006_3,Adr_F_006_4,Adr_F_021_1,Adr_F_021_2,...,Utr_F_006_3,Utr_F_006_4,Utr_F_021_1,Utr_F_021_2,Utr_F_021_3,Utr_F_021_4,Utr_F_104_1,Utr_F_104_2,Utr_F_104_3,Utr_F_104_4
Gene_id,,,,,,,,,,,,,,,,,,,,,
Vom2r3,4,3,1,2,2,1,3,6,4,7,...,0,2,2,1,1,3,0,2,0,0
LOC100909608,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
Vom2r6,27,33,36,34,37,78,64,75,49,57,...,0,0,0,0,0,1,0,3,0,1
Vom2r5,0,0,4,1,1,3,4,1,6,6,...,0,1,0,0,0,0,0,1,0,0
Raet1l,0,2,2,7,5,4,0,6,7,1,...,2,1,2,5,9,6,1,5,6,5
AABR07000109.1,0,0,0,1,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,2,0
AABR07000121.1,67,87,76,68,77,81,64,77,62,80,...,343,196,128,278,385,186,174,168,220,178
AABR07000137.1,0,0,0,0,0,3,1,0,0,2,...,4,0,0,7,3,2,3,2,2,0
AABR07000145.1,1,0,0,0,0,0,0,2,1,0,...,2,0,2,2,1,0,0,1,2,1


In [ ]:
full_count_lib = tmm_norm(full_count)
log_full_count = log_cpm(full_count, fulL_count_lib)

In [4]:
GO_df = pd.read_csv('./data/gene_association.rgd', sep='\t', skiprows=26, 
            names=['DB Object ID', 'DB Object Symbol', 'Qualifier', 'GO ID', 'DB:Reference', 'Evidence Code', 
                     'With or From', 'Aspect', 'DB Object Name', 'DB Object Synonym', 'DB Object Type', 'Taxon', 
                     'Date', 'Assigned By', 'Gene Product Form ID', 'NA'])

/n/scrb152/Software/Python/pyscrb152/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,10,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
genes

Index([u'Vom2r3', u'LOC100909608', u'Vom2r6', u'Vom2r5', u'Raet1l',
       u'AABR07000109.1', u'AABR07000121.1', u'AABR07000137.1',
       u'AABR07000145.1', u'Raet1d',
       ...
       u'AABR07022620.1', u'AABR07022674.1', u'AABR07022763.1',
       u'AABR07022303.1', u'AABR07022369.1', u'AABR07022926.1',
       u'AABR07046563.1', u'LOC100910067', u'AABR07045682.1',
       u'AABR07046248.1'],
      dtype='object', name=u'Gene_id', length=32140)

In [ ]:
from collections import defaultdict
GO_dict = defaultdict(list)
for gene in genes:
    GOs = GO_df[GO_df['DB Object Symbol'] == gene]
    #unique_ids = GOs['GO ID'].unique()
    #GO_dict.update(zip(unique_ids, [gene] * len(unique_ids)))
    #if gene == 'Vom2r5':
    #    break
    for key, GO in GOs.iterrows():
        if not (gene in GO_dict[GO['GO ID']]):
            GO_dict[GO['GO ID']].append(gene)
GO_dict

In [ ]:
import pickle
pickle.dump(GO_dict, 'GO_dict')

In [19]:
from collections import Counter
def genes_to_GO_IDs(GO_df, genes):
    GO_IDs = Counter()
    for g in genes:
        GOs = GO_df[GO_df['DB Object Symbol'] == g]
        GO_IDs.update(list(GOs['GO ID'].unique()))
    return GO_IDs
genes_to_GO_IDs(GO_df, ['Trpv5'])

Counter({'GO:0005262': 1,
         'GO:0005516': 1,
         'GO:0005887': 1,
         'GO:0006816': 1,
         'GO:0016324': 1,
         'GO:0031226': 1,
         'GO:0035809': 1,
         'GO:0046872': 1,
         'GO:0051262': 1,
         'GO:0055074': 1,
         'GO:0060402': 1,
         'GO:0070588': 1,
         'GO:1990035': 1})

In [20]:
mean_centered_full_counts = (full_count.T - full_count.mean(1)).T
cov_full_counts =(1./len(full_count.T))*mean_centered_full_counts.dot(mean_centered_full_counts.T)
cov_full_counts

Gene_id,Vom2r3,LOC100909608,Vom2r6,Vom2r5,Raet1l,AABR07000109.1,AABR07000121.1,AABR07000137.1,AABR07000145.1,Raet1d,...,AABR07022620.1,AABR07022674.1,AABR07022763.1,AABR07022303.1,AABR07022369.1,AABR07022926.1,AABR07046563.1,LOC100910067,AABR07045682.1,AABR07046248.1
Gene_id,,,,,,,,,,,,,,,,,,,,,
Vom2r3,8.993398,0.861367,19.656621,1.427187,-1.963320,-0.134082,39.252871,-4.490430,-0.707773,-3.355078,...,-0.012324,0.009883,-0.035215,0.0,0.001465,0.006934,-1.240566,4.214316,-0.028711,0.112168
LOC100909608,0.861367,0.532461,3.739043,0.242813,-0.410977,-0.006348,-6.030332,-0.574023,-0.116367,-0.662891,...,0.000566,-0.001836,0.004238,0.0,0.012012,-0.004395,0.007480,0.378770,-0.009180,0.021152
Vom2r6,19.656621,3.739043,340.558115,11.286406,-11.868926,-0.889893,-520.071260,-28.591699,-3.627559,-23.029883,...,0.038877,-0.029355,0.038213,0.0,-0.252588,0.509521,-5.355791,18.831416,-0.171777,0.096357
Vom2r5,1.427187,0.242813,11.286406,1.060000,-0.674062,-0.024219,-21.107969,-1.332813,-0.166562,-0.946875,...,0.003594,-0.002813,-0.006406,0.0,-0.021094,0.022656,-0.259219,1.395469,-0.010937,0.012656
Raet1l,-1.963320,-0.410977,-11.868926,-0.674062,48.991836,1.422559,437.562949,48.911914,7.412070,58.808984,...,0.053535,0.024102,0.049082,0.0,-0.093457,0.058887,1.381387,-1.295762,0.245508,-0.076816
AABR07000109.1,-0.134082,-0.006348,-0.889893,-0.024219,1.422559,0.291943,15.097607,1.677441,0.201270,1.988086,...,-0.000146,-0.001465,-0.000342,0.0,0.003467,0.004639,0.224951,-0.274951,0.005176,0.004443
AABR07000121.1,39.252871,-6.030332,-520.071260,-21.107969,437.562949,15.097607,22784.215615,470.361426,72.871816,768.520117,...,0.806377,2.671895,0.668838,0.0,-0.408838,-0.352979,16.082334,14.903916,0.303223,-1.703018
AABR07000137.1,-4.490430,-0.574023,-28.591699,-1.332813,48.911914,1.677441,470.361426,87.606836,9.574805,87.844141,...,0.157715,0.002148,-0.027832,0.0,-0.237793,0.034863,11.231738,-10.325488,0.263867,-0.106934
AABR07000145.1,-0.707773,-0.116367,-3.627559,-0.166562,7.412070,0.201270,72.871816,9.574805,2.343398,10.864453,...,0.022637,0.000117,-0.008848,0.0,-0.021777,0.000879,0.275254,-0.223379,0.041211,-0.014980


In [21]:
agg = {}
samples = list(full_count.columns)
for i in range(len(samples)/4):
    for s in samples[i*4:i*4+4]:
        agg[s] = samples[i * 4]
summarized = full_count.groupby(agg, axis=1).mean()
summarized.head()

,Adr_F_002_1,Adr_F_006_1,Adr_F_021_1,Adr_F_104_1,Adr_M_002_1,Adr_M_006_1,Adr_M_021_1,Adr_M_104_1,Brn_F_002_1,Brn_F_006_1,...,Thm_M_021_1,Thm_M_104_1,Tst_M_002_1,Tst_M_006_1,Tst_M_021_1,Tst_M_104_1,Utr_F_002_1,Utr_F_006_1,Utr_F_021_1,Utr_F_104_1
Gene_id,,,,,,,,,,,,,,,,,,,,,
Vom2r3,2.50,3.00,4.25,4.50,3.5,4.75,1.50,3.00,1.25,0.75,...,0.50,1.25,7.75,5.25,8.75,11.00,1.75,1.00,1.75,0.50
LOC100909608,0.25,0.25,0.25,0.25,1.5,0.75,0.50,1.25,0.25,0.25,...,0.50,0.25,1.50,0.75,3.50,0.50,0.00,0.00,0.00,0.25
Vom2r6,32.50,63.50,37.75,46.50,36.5,25.00,22.75,23.25,0.75,1.00,...,0.25,0.25,123.00,16.75,8.25,5.25,6.00,0.50,0.25,1.00
Vom2r5,1.25,2.25,3.50,1.50,1.5,1.50,0.75,1.25,0.00,0.00,...,0.50,0.25,3.25,2.25,3.00,1.50,0.25,0.25,0.00,0.25
Raet1l,2.75,3.75,3.00,1.75,3.0,4.25,1.50,2.75,1.75,1.00,...,10.00,12.00,6.00,5.00,3.25,2.75,1.50,2.25,5.50,4.25


In [ ]:
#for GO, gene_list in GO_dict:
#    
#    break
from collections import defaultdict
GO = 'GO:0030674'
gene_list = GO_dict[GO]
#print gene_list
not_in_set = [str(g) for g in full_count.index.values if g not in gene_list]
T = defaultdict(list)
not_T = defaultdict(list)
for t in gene_list:
    for in_T in gene_list:
        if t == in_T:
            continue
        T[gene].append(scistats.pearsonr(full_count.ix[t], full_count.ix[in_T]))
    for not_in_T in not_in_set:
        not_T[gene].append(scistats.pearsonr(full_count.ix[t], full_count.ix[not_in_T]))
#scistats.mannwhitneyu()

In [ ]:
T

In [ ]:
not_T

In [ ]:
x = 'AABR07046248.1'
scistats.mannwhitneyu(T[x], not_T[x])

In [51]:
RGD = pd.read_csv('ftp://ftp.rgd.mcw.edu/pub/data_release/GENES_RAT.txt', sep='\t', skiprows=74)
RGD

,GENE_RGD_ID,SYMBOL,NAME,GENE_DESC,CHROMOSOME_CELERA,CHROMOSOME_3.1,CHROMOSOME_3.4,FISH_BAND,START_POS_CELERA,STOP_POS_CELERA,...,ENSEMBL_ID,GENE_REFSEQ_STATUS,CHROMOSOME_5.0,START_POS_5.0,STOP_POS_5.0,STRAND_5.0,CHROMOSOME_6.0,START_POS_6.0,STOP_POS_6.0,STRAND_6.0
0,1595727,2463a1a2,class I gene fragment 2463,NaN,20,NaN,20,p12,43605,44306,...,NaN,INFERRED,20,5196932,5197633,-,20,3098274,3098975,-
1,1595726,2464a1,class I gene fragment 2464,NaN,20,NaN,20,p12,42617,42831,...,NaN,INFERRED,20,5195944,5196158,-,20,3097286,3097500,-
2,1595728,2461a1a2,class I gene fragment 2461,NaN,20,NaN,20,p12,45820,46493,...,NaN,INFERRED,20,5199147,5199820,-,20,3100489,3101162,-
3,1595729,2458a2,class I gene fragment 2458,NaN,20,NaN,20,p12,49408,49623,...,NaN,INFERRED,20,5202735,5202950,-,20,3104077,3104292,-
4,1594427,2331ex4-5,class I gene fragment 2331,NaN,20,NaN,20,p12,150959,151483,...,NaN,INFERRED,20,5330480,5331004,-,20,3232314,3232838,-
5,1594428,2310ex4-5,class I gene fragment 2310,NaN,20,NaN,20,p12,171896,172245,...,NaN,INFERRED,20,5351633,5351982,-,20,3253467,3253816,-
6,1594425,2361ex4-5,class I gene fragment 2361,NaN,20,NaN,20,p12,171797,172346,...,NaN,INFERRED,20,5297083,5297432,-,20,3198917,3199266,-
7,1594469,2509a1,class I gene fragment 2509,NaN,NaN,NaN,20,p12,NaN,NaN,...,NaN,INFERRED,20,5160942,5161150,-,20,3062284,3062492,-
8,1594457,2600a1a2,class I gene fragment 2600,NaN,NaN,NaN,20,p12,NaN,NaN,...,NaN,INFERRED,20,53977,54607,-,20,54530,55160,-
9,1594463,2539a1a2,class I gene fragment 2539,NaN,NaN,NaN,20,p12,NaN,NaN,...,NaN,INFERRED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
